# starting package

basic start

In [11]:
import torch
import matplotlib.pyplot as plt

from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from jcopdl.layers import linear_block
from jcopdl.callback import Callback, set_config
from tqdm.auto import tqdm

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Dataset & DataLoader

dataset image

In [ ]:
transform=transforms.Compose([
    transforms.ToTensor()
])

bs=0

train_dir='folder train'
train_set=datasets.ImageFolder(train_dir)
trainloader=DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=2)

test_dir='folder test'
test_set=datasets.ImageFolder(test-dir)
testloader=DataLoader(test_set, batch_size=bs, shuffle=True, num_workers=2) 

Dataset aksara

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from torch.utils.data import TensorDataset

random_state=42;

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)
X_train, X_val, y_train, y_test=train_test_split(X_train, y_train, test_size=0.25, stratify=y, random_state=random_state)

numeric=Pipeline([
    ('imputer',SimpleImputer(strategy="median")),
    ("scaler",MinMaxScaler())
])

categorical=Pipeline([
    ('imputer',SimpleImputer(strategy="most_frequent")),
    ("onehot",OneHotEncoder())
])

preprocessor=ColumnTransformer([
    ('numeric', numeric, ['column feature numeric']),
    ('categoric', categorical, ['column feature categorical'])
])

X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

X_train=torch.LongTensor(X_train).to(device)
X_test=torch.LongTensor(X_test).to(device)
y_train=torch.LongTensor(X_train).to(device)
y_test=torch.LongTensor(X_test).to(device)

bs=0

train_set=TensorDataset(X_train,y_train),
trainloader=DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=2)


train_set=TensorDataset(X_test,y_test),
trainloader=DataLoader(train_set, batch_size=bs)

# Arsitektur & Config

In [ ]:
Class CustomClassifier(nn.Module):
    def __init__(self, input_size, n1, n2, output_size, dropout=0.2):
        super().__init__()
        self.fc=nn.Sequential(
            nn.Flatten(),
            
            nn.Linear(input_size, n1),
            nn.ReLU(),
            nn.Dropout(dropout)
            
            nn.Linear(n1, n2),
            nn.ReLU(),
            nn.Dropout(dropout)
            
            nn.Linear(n2, output_size),
            nn.LogSoftmax(1),
        )
        
    def forward(self,x):
        return self.fc(x)

In [9]:
config= set_config({
    "input_size":input_size,
    "n1":n1,
    "n2":n2,
    "output_size": len(train_set.classes),
    "dropout":dropout
})

# Training Preparation ( MCOC )

In [ ]:
model=CustomClassifier(config.input_size, config.n1, config.n2, config.output_size, config.dropout).to(device)
citerion=nn.NLLLoss()
optimizer=optim.AdamW(model.parameters(), lr=0.001)
callback=Callback(model, config, outdir="your directory", save_every=20, early_stop_patience=5, plot_every= 20)

# Training

In [ ]:
def loop_fn(mode, dataset, dataloader, model, citerion, optimizer, device):
    if mode =='train':
        model.train()
    elif mode =='test':
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target=feature.to(device), target.to(device)
        output = model(feature)
        loss = citerion(output, target)
        
        if mode == 'train':
            loss.bacward()
            optimizer.step()
            optimizer.zero_grad()
        
        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    return cost, acc    

In [ ]:
while True:
    train_cost, train_score = loop_fn("train", train_set, trainloader, model, citerion, optimizer, device)
    with torch.no_grad():
        test_cost, test_score = loop_fn("test", test_set, testloader, model, citerion, optimizer, device)
        
    callback.log(train_cost, test_cost, train_score, test_score)
    callback.save_checkpoint()
    callback.cost_runtime_plotting()
    callback.score_runtime_plotting()
    
    if callback.early_stopping(model, monitor="score_test"):
        callback.plot_cost()
        callback.plot_score()
        break

# Load Model

In [ ]:
weights=torch.load("your directory/weights_best.pth", map_location="cpu")
config=torch.load("your directory/configs.pth", map_location="cpu")

model=CustomClassifier(config.input_size, config.n1, config.n2, config.output_size, config.dropout)
model.load_state_dict(weights)
model=model.to(device)

# Predict

In [ ]:
feature, target = next(iter(testloader))
feature, target = feature.to(device), target.to(device)

with torch.no_grad():
    model.aval()
    output = model(feature)
    predictions = output.argmax(1)
[label2cat[prediction] for prediction in predictions]

# Visualisasi Predictions ( image dataset ) 

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(25, 25))
for img, label, prediction, ax in zip(feature, target, predictions, axes.flatten()):
    ax.imshow(img.view(28, 28).cpu(), cmap='')
    font= {'color' : 'r'} if label != pred else {'color' : 'g'}
    label, prediction = label2cat[label.item()], label2cat[pred.item()]
    ax.set_title(f" Label: {label} | Pred: {prediction}",fontdict = font);
    ax.axis('off')''